In [1]:
import pandas as pd
import numpy as np
import mysql.connector as sql
import breakinghits_script as bh
import networkx as nx
from networkx.algorithms import bipartite as bi
import heapq
import operator
import pickle
import flask

In [ ]:
def recommendations():
    recommendation_dict = {}
    rate = bh.get_ratings()
    user_list = list(bh.pull_dataframe('select id from users')['id'])
    song_list = set(list(bh.pull_dataframe('select id from user_musics')['id']))
    spotlights = bh.get_spotlights()
    saves = bh.get_saves()
    sparse_spots = bh.get_sparse(spotlights) 
    sparse_saves = bh.get_sparse(saves)
    svd = bh.bh_model(rate)
    for i in user_list:
        implicit, user_spot_save = bh.ensemble_implicit_recommendations(sparse_spots, sparse_saves,i,drop=False)
        predictions = bh.bh_predict(svd, i, rate)
        explicit = bh.bh_top_recs(predictions)
        implicit['song_id'] = implicit.index.values
        implicit.columns = ['spotlight_saved', 'Song_ID']
        total = implicit.merge(explicit, how='left',on='Song_ID').replace(np.nan, 0)
        total['final_sum'] = total['spotlight_saved'] + total['rating']
        total.index = total['Song_ID']
        user_rated = np.unique(np.append(np.array(rate[rate['user_id'] == i]['user_music_id']),(user_spot_save)))
        recommendation_dict[i] = list(total.drop(user_rated, errors='ignore').nlargest(10,['final_sum'])['Song_ID'])
    return(recommendation_dict)

In [2]:
recommendations()

NameError: name 'recommendations' is not defined

In [3]:
pickle_in = open("data/recommendation.pkl","rb")
rec_dict = pickle.load(pickle_in)